## ETA

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import BertConfig, BertModel
from tqdm import tqdm
from matplotlib import pyplot as plt
import torch.nn.functional as F


d:\anaconda\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def process_data(file_path, output_path):
    # 读取数据

    df = pd.read_csv(file_path)

    df['time'] = pd.to_datetime(df['time'])
    base_time = df['time'].min()
    
    if os.path.exists(output_path):
        print(f"{output_path} already exists. Skipping processing.")
        return base_time


    df['time_offset'] = (df['time'] - base_time).dt.total_seconds()

    df[['longitude', 'latitude']] = df['coordinates'].apply(lambda x: eval(x) if pd.notna(x) else [None, None]).tolist()
    
    # 转换
    df[['longitude', 'latitude']] = df.apply(lambda row: pd.Series(tbd.gcj02towgs84(row['longitude'], row['latitude'])), axis=1)

    df['holiday'] = df['time'].apply(lambda x: 1 if (x.month == 10 and 1 <= x.day <= 7) else 0)

    weather_df = pd.read_csv('./data/weather.csv')


    weather_df['Date'] = pd.to_datetime(weather_df['Date'])
    df['date'] = df['time'].dt.date
    df['date'] = pd.to_datetime(df['date'])

    # 合并轨迹数据和天气数据
    df = pd.merge(df, weather_df, left_on='date', right_on='Date', how='left')

    df.drop(columns=['Date','Day','date'], inplace=True)

    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

    # 保存处理后的数据
    df.to_csv(output_path, index=False)
    return base_time

base_time = process_data('./data/eta_task.csv', './product_data/task3_todo.csv')
print(base_time)


./product_data/task3_todo.csv already exists. Skipping processing.
2018-09-30 15:50:58+00:00


In [3]:
df = pd.read_csv('./product_data/task3_todo.csv')

cols_to_fill = ['High Temp', 'Low Temp', 'Rain', 'Wind Force']

ref_df = df.groupby('trajectory_id')[cols_to_fill].first().reset_index()
holiday_ref_df = df.groupby('trajectory_id')['holiday'].first().reset_index()

merged = pd.merge(df, ref_df, on='trajectory_id', suffixes=('', '_ref'))
merged = pd.merge(merged, holiday_ref_df, on='trajectory_id', suffixes=('', '_holiday_ref'))

for col in cols_to_fill:
    merged[col] = merged[col].fillna(merged[col + '_ref'])

merged['holiday'] = merged['holiday_holiday_ref']

cols_to_drop = [col + '_ref' for col in cols_to_fill] + ['holiday_holiday_ref']
merged.drop(columns=cols_to_drop, inplace=True)

df_filled = merged
df_filled.to_csv('./product_data/task3_todo.csv', index=False)

In [4]:
if not os.path.exists('models'):
    os.makedirs('models')

df = pd.read_csv('./product_data/task4_train_data.csv') # revise after task4

df = df.drop(['type', 'time', 'coordinates'], axis=1)

unique_ids = df['trajectory_id'].unique()

train_ids, test_ids = train_test_split(
    unique_ids,
    test_size=0.2,
    random_state=114514
)

train_df = df[df['trajectory_id'].isin(train_ids)].copy()
test_df = df[df['trajectory_id'].isin(test_ids)].copy()

In [5]:
continuous_cols = ['longitude', 'latitude']

discrete_cols = ['holiday','Rain', 'High Temp', 'Low Temp', 'Wind Force']

target_col = ['time_offset']

# 计算训练集的均值和标准差
target_mean = train_df['time_offset'].mean()
target_std = train_df['time_offset'].std()

# 使用训练集的参数标准化训练集
train_df['time_offset'] = (train_df['time_offset'] - target_mean) / target_std

# 使用相同的参数标准化测试集
test_df['time_offset'] = (test_df['time_offset'] - target_mean) / target_std


In [6]:
# 标准化连续特征
scaler = StandardScaler()

# 拟合并转换训练集
train_df[continuous_cols] = scaler.fit_transform(train_df[continuous_cols])

# 仅转换测试集
test_df[continuous_cols] = scaler.transform(test_df[continuous_cols])

In [7]:
# 编码离散特征
label_encoders = {}
for col in discrete_cols:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    test_df[col] = le.transform(test_df[col])
    label_encoders[col] = le

In [8]:
# 将数据按trajectory_id分组
def group_by_trajectory(df):
    grouped = df.groupby('trajectory_id')
    sequences = []
    for _, group in grouped:
        group_data = group.sort_values(by='trajectory_id')
        sequences.append({
            'features': group[continuous_cols + discrete_cols].values,
            'target': group[target_col].values
        })
    return sequences

train_sequences = group_by_trajectory(train_df)
test_sequences = group_by_trajectory(test_df)

In [9]:
max_encoder_len = 30 
max_decoder_len = 15 

def process_sequences(sequences, max_enc_len, max_dec_len):
    encoder_inputs = []
    decoder_inputs = []
    targets = []
    encoder_masks = []
    decoder_masks = []

    for seq in sequences:
        features = seq['features']  # [seq_len, feature_dim]
        target = seq['target']      # [seq_len, 1]

        enc_input = features[:max_enc_len]
        encoder_inputs.append(torch.tensor(enc_input, dtype=torch.float32))

        dec_input = features[:max_dec_len]  
        decoder_inputs.append(torch.tensor(dec_input, dtype=torch.float32))

        target_seq = target[:max_dec_len]
        targets.append(torch.tensor(target_seq, dtype=torch.float32))

        enc_mask = torch.zeros(max_enc_len, dtype=torch.int64)
        enc_mask[:len(enc_input)] = 1
        encoder_masks.append(enc_mask)

        dec_mask = torch.zeros(max_dec_len, dtype=torch.int64)
        dec_mask[:len(dec_input)] = 1
        decoder_masks.append(dec_mask)

    encoder_inputs = pad_sequence(encoder_inputs, batch_first=True, padding_value=0.0)  # [B, T_enc, F]
    decoder_inputs = pad_sequence(decoder_inputs, batch_first=True, padding_value=0.0)  # [B, T_dec, F]
    targets = pad_sequence(targets, batch_first=True, padding_value=0.0)                # [B, T_dec]

    encoder_masks = torch.stack(encoder_masks)  # [B, T_enc]
    decoder_masks = torch.stack(decoder_masks)  # [B, T_dec]

    return encoder_inputs, decoder_inputs, targets, encoder_masks, decoder_masks



In [10]:
class TimeSeriesDataset(Dataset):
    def __init__(self, encoder_inputs, decoder_inputs, targets, encoder_masks, decoder_masks):
        self.encoder_inputs = encoder_inputs  # Encoder输入
        self.decoder_inputs = decoder_inputs  # Decoder输入 (通常是目标序列的历史值)
        self.targets = targets                # 目标序列
        self.encoder_masks = encoder_masks    # Encoder mask
        self.decoder_masks = decoder_masks    # Decoder mask

    def __len__(self):
        return len(self.encoder_inputs)

    def __getitem__(self, idx):
        return {
            'encoder_inputs': self.encoder_inputs[idx].clone().detach().to(dtype=torch.float32),
            'decoder_inputs': self.decoder_inputs[idx].clone().detach().to(dtype=torch.float32),
            'targets': self.targets[idx].clone().detach().to(dtype=torch.float32),
            'encoder_masks': self.encoder_masks[idx].clone().detach().to(dtype=torch.int64),
            'decoder_masks': self.decoder_masks[idx].clone().detach().to(dtype=torch.int64)
        }

In [11]:
class Transformer_EncoderDecoder_2(nn.Module):
    def __init__(self, feature_dim, d_model=128, num_layers=6, num_heads=4):
        super(Transformer_EncoderDecoder_2, self).__init__()
        # Encoder配置
        self.encoder_config = BertConfig(
            hidden_size=d_model,
            num_hidden_layers=num_layers,
            num_attention_heads=num_heads,
            intermediate_size=4 * d_model,
            max_position_embeddings=128,
            hidden_dropout_prob=0.1,
            attention_probs_dropout_prob=0.05
        )
        self.encoder = BertModel(self.encoder_config)

        # Decoder配置
        self.decoder_config = BertConfig(
            hidden_size=d_model,
            num_hidden_layers=num_layers,
            num_attention_heads=num_heads,
            intermediate_size=4 * d_model,
            max_position_embeddings=128,
            hidden_dropout_prob=0.03,
            attention_probs_dropout_prob=0.05,
            is_decoder=True,
            add_cross_attention=True  # 启用交叉注意力
        )
        self.decoder = BertModel(self.decoder_config)

        # 输入映射到 Transformer 的 d_model
        self.input_projection = nn.Linear(feature_dim, d_model)

        # 输出预测层
        self.fc = nn.Linear(d_model, 1)

    def forward(self, encoder_inputs, decoder_inputs, encoder_masks, decoder_masks):
        # Encoder阶段
        encoder_inputs = self.input_projection(encoder_inputs)  # [batch_size, seq_len, d_model]
        encoder_outputs = self.encoder(inputs_embeds=encoder_inputs, attention_mask=encoder_masks)

        # Decoder阶段
        decoder_inputs = self.input_projection(decoder_inputs)  # [batch_size, seq_len, d_model]
        decoder_outputs = self.decoder(
            inputs_embeds=decoder_inputs,
            attention_mask=decoder_masks,
            encoder_hidden_states=encoder_outputs.last_hidden_state,
            encoder_attention_mask=encoder_masks
        )

        # 输出预测
        increments_logits = self.fc(decoder_outputs.last_hidden_state)
        # （2）确保增量为正
        increments = F.softplus(increments_logits) + 1e-9
        # （3）在 seq_len 维度上做累加
        predictions_cumsum = torch.cumsum(increments, dim=1) 
        
        # 返回去掉最后一维的结果 => [batch_size, seq_len]
        return predictions_cumsum.squeeze(-1)
    
class TransformerEncoderDecoder(nn.Module):
    def __init__(self, feature_dim, d_model=128, num_layers=6, num_heads=8):
        super(TransformerEncoderDecoder, self).__init__()
        # Encoder配置
        self.encoder_config = BertConfig(
            hidden_size=d_model,
            num_hidden_layers=num_layers,
            num_attention_heads=num_heads,
            intermediate_size=4 * d_model,
            max_position_embeddings=512,
            hidden_dropout_prob=0.1,
            attention_probs_dropout_prob=0.1
        )
        self.encoder = BertModel(self.encoder_config)

        # Decoder配置
        self.decoder_config = BertConfig(
            hidden_size=d_model,
            num_hidden_layers=num_layers,
            num_attention_heads=num_heads,
            intermediate_size=4 * d_model,
            max_position_embeddings=512,
            hidden_dropout_prob=0.1,
            attention_probs_dropout_prob=0.1,
            is_decoder=True,
            add_cross_attention=True  # 启用交叉注意力
        )
        self.decoder = BertModel(self.decoder_config)

        # 输入映射到 Transformer 的 d_model
        self.input_projection = nn.Linear(feature_dim, d_model)

        # 输出预测层
        self.fc = nn.Linear(d_model, 1)

    def forward(self, encoder_inputs, decoder_inputs, encoder_masks, decoder_masks):
        # Encoder阶段
        encoder_inputs = self.input_projection(encoder_inputs)  # [batch_size, seq_len, d_model]
        encoder_outputs = self.encoder(inputs_embeds=encoder_inputs, attention_mask=encoder_masks)

        # Decoder阶段
        decoder_inputs = self.input_projection(decoder_inputs)  # [batch_size, seq_len, d_model]
        decoder_outputs = self.decoder(
            inputs_embeds=decoder_inputs,
            attention_mask=decoder_masks,
            encoder_hidden_states=encoder_outputs.last_hidden_state,
            encoder_attention_mask=encoder_masks
        )

        # 输出预测
        predictions = self.fc(decoder_outputs.last_hidden_state)  # [batch_size, seq_len, 1]
        return predictions.squeeze(-1)

In [12]:
def offset_to_hour(data,predictions_list,i):
    temp = data.groupby('trajectory_id').first()
    temp = pd.to_datetime(temp['time'])

    temp_hour = temp.dt.hour.tolist()
    temp_minute = temp.dt.minute.tolist()

    # 将预测值看作秒数，加到基准时间 base_time
    todo = base_time + pd.Timedelta(seconds=float(predictions_list[0]))

    # 让输出时间的小时固定为和 ref_time 相同
    new_hour = temp_hour[i]

    # 如果输出时间的分钟大于 ref_time 的分钟，则加一小时
    if todo.minute > temp_minute[i] + 10:
        new_hour += 1

    # 用 replace 更新小时
    # 注意：replace 不改变日期，但会改变小时，可能需要考虑跨日情况
    # 如果 new_hour >= 24，需要额外加一天，这里仅作简单示例
    if new_hour >= 24:
        # 加一天，并让小时在 0-23 之间
        todo = todo + pd.Timedelta(days=1)
        new_hour = new_hour % 24

    todo = todo.replace(hour=new_hour)

    # 最后输出你想要的格式
    time_str = todo.strftime('%Y-%m-%d %H:%M:%S.%f%z')
    return time_str

In [13]:
def custom_loss(preds, targets, lambda_=0.07):
    """
    preds:  形状 [batch_size, seq_len], 模型输出的预测序列
    targets:[batch_size, seq_len], 真实标签
    lambda_: 惩罚系数
    """
    # 1) mae
    base_loss = F.mse_loss(preds, targets)
    # 2) 惩罚项：
    increments = preds[:, 1:] - preds[:, :-1]  # [batch_size, seq_len-1]
    
    
    penalty = F.relu(increments - 1e-7)        
    
    penalty_value = penalty.sum()

    # 3) 合并最终损失
    total_loss = base_loss + lambda_ * penalty_value
    
    return total_loss, penalty_value

In [14]:
encoder_inputs, decoder_inputs, targets, encoder_masks, decoder_masks = process_sequences(
    train_sequences, max_encoder_len, max_decoder_len
)
test_encoder_inputs, test_decoder_inputs, test_targets, test_encoder_masks, test_decoder_masks = process_sequences(
    test_sequences, max_encoder_len, max_decoder_len
)

dataset = TimeSeriesDataset(encoder_inputs, decoder_inputs, targets, encoder_masks, decoder_masks)
test_dataset = TimeSeriesDataset(test_encoder_inputs, test_decoder_inputs, test_targets, test_encoder_masks, test_decoder_masks)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# 初始化模型
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TransformerEncoderDecoder(feature_dim=7, d_model=128, num_layers=2, num_heads=4).to(device)
epochs = 100
evaluation_interval = 5

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs / 4, eta_min=2e-6)

train_losses = []
test_rmse = []
test_mae = []
evaluation_epochs = []

model.to(device)
model.train()

def compute_rmse(preds, targets):
    return torch.sqrt(nn.MSELoss()(preds, targets)).item()

# Function to compute MAE
def compute_mae(preds, targets):
    return nn.L1Loss()(preds, targets).item()

In [15]:

for epoch in range(epochs):
    epoch_loss = 0.0
    with tqdm(dataloader, desc=f"Epoch {epoch + 1}/{epochs}", unit="batch") as progress_bar:
        for batch in progress_bar:
            encoder_inputs = batch['encoder_inputs'].to(device)
            decoder_inputs = batch['decoder_inputs'].to(device)
            targets = batch['targets'].to(device)
            targets = targets.squeeze(-1)  # Remove the last dimension
            encoder_masks = batch['encoder_masks'].to(device)
            decoder_masks = batch['decoder_masks'].to(device)

            optimizer.zero_grad()
            predictions = model(encoder_inputs, decoder_inputs, encoder_masks, decoder_masks)
            
            loss, penalty = custom_loss(predictions, targets, lambda_=0.07)
            loss.backward()
            
            optimizer.step()
            scheduler.step()

            epoch_loss += loss.item()

            progress_bar.set_postfix(loss=loss.item())

    avg_train_loss = epoch_loss / len(dataloader)
    train_losses.append(avg_train_loss)
    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {avg_train_loss:.4f}, Penalty: {penalty:.7f}")

    if (epoch + 1) % evaluation_interval == 0:
        model.eval()  
        with torch.no_grad():
            total_rmse = 0.0
            total_mae = 0.0
            total_samples = 0

            for batch in test_dataloader:
                encoder_inputs = batch['encoder_inputs'].to(device)
                decoder_inputs = batch['decoder_inputs'].to(device)
                targets = batch['targets'].to(device)
                targets = targets.squeeze(-1)
                encoder_masks = batch['encoder_masks'].to(device)
                decoder_masks = batch['decoder_masks'].to(device)

                predictions = model(encoder_inputs, decoder_inputs, encoder_masks, decoder_masks)

                predictions = predictions.view(-1)

                targets = targets.view(-1)

                rmse = compute_rmse(predictions, targets)
                mae = compute_mae(predictions, targets)

                total_rmse += rmse * targets.size(0)
                total_mae += mae * targets.size(0)
                total_samples += targets.size(0)

            avg_rmse = total_rmse / total_samples
            avg_mae = total_mae / total_samples

            test_rmse.append(avg_rmse)
            test_mae.append(avg_mae)
            evaluation_epochs.append(epoch + 1)
            current_lr = optimizer.param_groups[0]['lr']

            print(f"--- Evaluation after Epoch {epoch + 1} ---")
            print(f"Test RMSE: {avg_rmse:.4f}, Test MAE: {avg_mae:.4f}, Learning Rate: {current_lr:.6f}")

        model.train()




Epoch 1/100:  68%|██████▊   | 226/333 [00:05<00:02, 41.41batch/s, loss=0.998]


KeyboardInterrupt: 

In [ ]:

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(range(1, epochs + 1), train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(evaluation_epochs, test_rmse, label='Test RMSE', marker='o')
plt.plot(evaluation_epochs, test_mae, label='Test MAE', marker='s')
plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.title('Test Metrics Over Epochs')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()
torch.save(model.state_dict(), './models/task3.pth')

In [ ]:
state_dict = torch.load('./models/3.pth')
model = TransformerEncoderDecoder(feature_dim=7, d_model=128, num_layers=3, num_heads=4).to(device)
model.load_state_dict(state_dict)
model.eval()
df = pd.read_csv('./product_data/task3_todo.csv')
df[continuous_cols] = scaler.transform(df[continuous_cols])

discrete_cols = ['holiday','Rain', 'High Temp', 'Low Temp', 'Wind Force']
for col in discrete_cols:
    df[col] = le.fit_transform(df[col])

fill_sequences = group_by_trajectory(df)

fill_encoder_inputs, fill_decoder_inputs, fill_targets, fill_encoder_masks, fill_decoder_masks = process_sequences(
    fill_sequences, max_encoder_len, max_decoder_len
)

fill_dataset = TimeSeriesDataset(fill_encoder_inputs, fill_decoder_inputs, fill_targets, fill_encoder_masks, fill_decoder_masks)
fill_dataloader = DataLoader(fill_dataset, batch_size=1, shuffle=False)

j = 0
times = []
ids = []
for batch in fill_dataloader:
    encoder_inputs = batch['encoder_inputs'].to(device)
    decoder_inputs = batch['decoder_inputs'].to(device)
    targets = batch['targets'].to(device).squeeze(-1)
    encoder_masks = batch['encoder_masks'].to(device)
    decoder_masks = batch['decoder_masks'].to(device)

    # 模型预测
    predictions = model(encoder_inputs, decoder_inputs, encoder_masks, decoder_masks)
    predictions = predictions * target_std + target_mean
    predictions_lists = predictions.tolist()

    # 遍历每条预测
    id = 26628 + j
    for i in range(len(predictions_lists)): 
        times.append(offset_to_hour(df, predictions_lists[i], i))
        ids.append(id)
    j += 1

df = pd.DataFrame({
    'id': ids,
    'time': times,
})

df.to_csv('./results/task3_results.csv', index=False)

print("DataFrame 已成功输出到 'output.csv' 文件中")

DataFrame 已成功输出到 'output.csv' 文件中
